# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

sum =0 

for sqrt_value in generator:
    print(sqrt_value)
    sum += sqrt_value
    
print("Sum of square roots from 1 to", limit, "is", sum)



1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
Sum of square roots from 1 to 13 is 32.854555867161245


- Q1 Answer is : Sum of square roots from 1 to 5 is 8.382332347441762
- Q2 Answer is : 13.element is 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**





In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


## Load to duckdb

In [3]:
import dlt
import duckdb

# define a pipeline
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')



info = pipeline.run(people_1(),
					table_name="people",
					write_disposition="replace")

info_2 = pipeline.run(people_2(),
					table_name="people",
					write_disposition="append")

print(info)

print('*' * 50)

print('Info 2:', info_2)


Pipeline dlt_ipykernel_launcher load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\hamit\Desktop\repos\DE-zoomcamp\workshops\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708285237.8742626 is LOADED and contains no failed jobs
**************************************************
Info 2: Pipeline dlt_ipykernel_launcher load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\hamit\Desktop\repos\DE-zoomcamp\workshops\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708285238.532123 is LOADED and contains no failed jobs


In [4]:
pipeline.pipeline_name, pipeline.dataset_name

('dlt_ipykernel_launcher', 'people')

In [5]:
# show the outcome

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))



people = conn.sql("SELECT * FROM people").df()

sum_ages = people['age'].sum()

print("Sum of all ages of people:", sum_ages)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_2            │
└─────────────────────┘

Sum of all ages of people: 353


In [6]:
people.head()

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708285237.8742626,RHR/slwsMMg6Yg,None
1,2,Person_2,27,City_A,1708285237.8742626,peV2QKvSRuRupQ,None
2,3,Person_3,28,City_A,1708285237.8742626,p/zd1HEzR4b0Pw,None
3,4,Person_4,29,City_A,1708285237.8742626,mjNuV64NkbNHjA,None
4,5,Person_5,30,City_A,1708285237.8742626,XI1yOFtzL9ofdQ,None


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [7]:
info = pipeline.run(people_1(),
					table_name="people_2",
					write_disposition="replace",
                    primary_key="id")

info_2 = pipeline.run(people_2(),
					table_name="people_2",
					write_disposition="merge",
					primary_key="id")

print(info)

print('*' * 50)

print('Info 2:', info_2)


Pipeline dlt_ipykernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\hamit\Desktop\repos\DE-zoomcamp\workshops\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708285239.2114303 is LOADED and contains no failed jobs
**************************************************
Info 2: Pipeline dlt_ipykernel_launcher load step completed in 0.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\hamit\Desktop\repos\DE-zoomcamp\workshops\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708285239.7783928 is LOADED and contains no failed jobs


In [8]:
# show the outcome

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))



people_2 = conn.sql("SELECT * FROM people_2").df()

sum_ages_2 = people_2['age'].sum()

print("Sum of all ages of people:", sum_ages_2)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_2            │
└─────────────────────┘

Sum of all ages of people: 266


In [9]:
people_2

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708285239.2114303,KFTeqmKpr3k8TQ,None
1,2,Person_2,27,City_A,1708285239.2114303,2Pn46xSfHeek3A,None
2,5,Person_5,35,City_B,1708285239.7783928,hPJKDCNGxvFhaQ,Job_5
3,7,Person_7,37,City_B,1708285239.7783928,SVyFXPGi0BWjkg,Job_7
4,4,Person_4,34,City_B,1708285239.7783928,9ty5sKovQq7/cQ,Job_4
5,8,Person_8,38,City_B,1708285239.7783928,qBC2hmznQFd6xg,Job_8
6,3,Person_3,33,City_B,1708285239.7783928,eGV0hoxINpM8wQ,Job_3
7,6,Person_6,36,City_B,1708285239.7783928,o+u2bw7pbFVY2A,Job_6


# Solution: First make sure that the following modules are installed:

In [10]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

UsageError: Line magic function `%%capture` not found.


In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX